### Carregar pesos de GPT2 Para modelo scratch

O objetivo deste notebook é carregar os pesos fornecidos pelo modelo pré-treinado GPT2 a uma instância de um modelo baseado no GPT criado do 0.

Iremos utilizar os códigos previamente construídos nos capítulos anteriores do livro Build a Large Language Model (From Scratch), onde aqui serão apresentados apenas os códigos referentes a seção 5.5.

In [6]:
!pip install tensorflow>=2.15.0

^C


In [1]:
!pip install tensorflow>=2.15.0

O arquivo j� est� sendo usado por outro processo.


In [5]:
from gpt_download import download_and_load_gpt2
settings, params = download_and_load_gpt2(
    model_size="124M", models_dir="gpt2"
)

ModuleNotFoundError: No module named 'tensorflow'